# Mask R-CNN - Inspect Lego Anchors

Inspect and visualize achnors loading and pre-processing code.

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

%matplotlib inline

%load_ext autoreload
%autoreload 2

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.model import compute_backbone_shapes
from samples.lego import lego



## Load Dataset

In [ ]:
# Configurations are defined in lego.py
config = lego.LegoConfig()
LEGO_DIR = os.path.join(ROOT_DIR, "datasets/lego")

# Load dataset
dataset = lego.LegoDataset()
dataset.load_lego(LEGO_DIR, "eval")

# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

## Display Samples

Load and display images and masks.

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset.image_ids, 2)

for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

## Anchors

In [ ]:
# Generate Anchors
if config.USE_LRPN:
    anchors = utils.generate_lrpn_anchors(config.IMAGE_SHAPE, config.LRPN_NETWORK_TYPE)
else:
    backbone_shapes = compute_backbone_shapes(config, config.IMAGE_SHAPE)
    anchors = utils.generate_pyramid_anchors(
        config.RPN_ANCHOR_SCALES,
        config.RPN_ANCHOR_RATIOS,
        backbone_shapes,
        config.BACKBONE_STRIDES,
        config.RPN_ANCHOR_STRIDE)


print("Image shape is:" + str(config.IMAGE_SHAPE))

# Print summary of anchors
print("Count: ", anchors.shape[0])

In [ ]:
## Visualize anchors of one cell at the center of the feature map of a specific level

# Load and draw random image
image_id = np.random.choice(dataset.image_ids, 1)[0]
image, image_meta, _, _, _ = modellib.load_image_gt(dataset, config, image_id)
fig, ax = plt.subplots(1, figsize=(20, 20))
ax.imshow(image)

sample_anchors = random.sample(list(anchors),k=50)
colors = visualize.random_colors(len(sample_anchors))

# Draw anchors. Brightness show the order in the array, dark to bright.
for i, rect in enumerate(sample_anchors):
    y1, x1, y2, x2 = rect
    w = x2-x1
    h = y2-y1

    #if w < 192 and w > 96:
    p = patches.Rectangle((x1, y1), w, h, linewidth=2, facecolor='none', edgecolor=colors[i])
    ax.add_patch(p)
